# Tests of context compression

In [14]:
import time
from wattelse.chatbot.backend.rag_backend import RAGBackEnd


In [2]:
DOC_REPO = "test_jp"

# Required configurations

if executed on a standard PC, be sure that :
-  `embedding_api.cfg` is properly configured (ex. that it contains a pointer to the "right" host, for example `host=10.132.6.110`


In [3]:
backend = RAGBackEnd(DOC_REPO)

2024-08-28 11:53:14.751 | INFO     | wattelse.chatbot.backend.rag_backend:__init__:129 - [Group: test_jp] Initialization of chatbot backend


In [4]:
question_list_conso = {
    "customer":  "Quel est le nom du client ?",
    "address": "Quelle est l'adresse du site de l'installation à raccorder (extraites seulement les valeurs d'adresse) ?",
    "date": "Quelle est la date de mise à disposition du raccordement souhaitée ? Répondre uniquement sous la forme JJ/MM/AAAA.",
    "request_pracc": "Quelle est la demande en puissance du client ? Quelle est la puissance maximale de soutirage demandée ?",
    "request_pcc": "Quelle est la puissance de court circuit ? Si l'information n'est pas fournie, répondez 'Je ne sais pas'",
    "request_additional_power_supply": "Quelles est la puissance de l’alimentation supplémentaire n°1 demandée ? Si l'information n'est pas fournie, répondez 'Je ne sais pas'",
}

question_list_prod = {
    "customer":  "Quel est le nom du client ?",
    "address": "Quelle est l'emplacement du point de raccordement (extraites seulement les valeurs d'adresse) ?",
    "lat_lon": "Quelle sont la longitude et la latitude du point de raccordement ? Répondre uniquement au format json.",
    "date":  "Quelle est la date de mise à disposition du raccordement souhaitée ? Répondre uniquement sous la forme JJ/MM/AAAA.",
    "request_pracc": "Quelle est la demande en puissance du client ?",
    "request_pinjection": "Quelle est la puissance de raccordement à l'injection demandée ?",
    "request_psoutirage": "Quelle est la puissance maximale de soutirage demandée ?",
}

In [5]:
fiche_D1A = "20230921 - Fiche D1 - EE France - Projet La Mottelette - Saint-Thibault (60).pdf"
fiche_D1B = "Demande d'Etude Explo - HDF.pdf"
fiche_D1C = "QairPE-H2-Tréport-Fiche D1-30MW.docx"

In [6]:
def pretty_print(answer):
    s = f"\n{'*' * 100}\nANSWER\n\n" +answer["answer"] + f"\n{'*' * 100}\n"
    s += f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d["content"] for i, d in enumerate(answer["relevant_extracts"])]
        )
    
    return s

In [7]:
backend.get_available_docs()

['20230921 - Fiche D1 - EE France - Projet La Mottelette - Saint-Thibault (60).pdf',
 "Demande d'Etude Explo - HDF.pdf",
 'QairPE-H2-Tréport-Fiche D1-30MW.docx']

In [8]:
# https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/
# https://blog.langchain.dev/improving-document-retrieval-with-contextual-compression/

In [22]:
def query_doc(question_list, fiche_D1):
    
    responses = {}
    start = time.time()
    
    for key, q in question_list.items():
        answer = backend.query_rag(q, selected_files=[fiche_D1])
        responses[key] = pretty_print(answer)
        print(responses[key])
    
    print(f"\n\n*** Execution time: {time.time() - start} seconds")
   # return responses

# Tests without context compression

In [19]:
backend.use_context_compression = False


In [20]:
question_list = {
    "customer":  "Quel est le nom du client ?",
    "address": "Quelle est l'adresse du site de l'installation à raccorder (extraites seulement les valeurs d'adresse) ?",
    "request_pracc": "Quelle est la demande en puissance du client ? Quelle est la puissance maximale de soutirage demandée ?",
}

In [21]:
query_doc(question_list, fiche_D1A)

2024-08-28 14:24:27.408 | DEBUG    | wattelse.chatbot.backend.rag_backend:query_rag:371 - Calling RAG chain for question : "Quel est le nom du client ?"...
2024-08-28 14:24:27.440 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:38 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2024-08-28 14:24:27.443 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:39 - Computing embeddings...
2024-08-28 14:24:27.525 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:43 - Computing embeddings done
2024-08-28 14:24:28.128 | DEBUG    | wattelse.chatbot.backend.rag_backend:query_rag:371 - Calling RAG chain for question : "Quelle est l'adresse du site de l'installation à raccorder (extraites seulement les valeurs d'adresse) ?"...
2024-08-28 14:24:28.143 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:38 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2024-08-28 14:24:28.147


****************************************************************************************************
ANSWER

Le nom du client est European Energy France.
****************************************************************************************************
Document 1:

INFORMATIONS SUR LE DEMANDEUR DU RACCORDEMENT Nom de la société : EUROPEAN ENERGY FRANCE n° SIREN : 913794707 Nom et fonction de l’interlocuteur : Emmanuel Fourmaux - Responsable Développement Régional Adresse Numéro / rue 5 PLACE DES COUREAULEURS Code postal/ ville 17000 LA ROCHELLE Pays FRANCE Téléphone/fax +33 (0)6 70 29 51 71 Courriel efo@europeanenergy.com Le demandeur est Producteur Demandeur du raccordement agissant en vertu d’un acte de désignation par le Groupement multi-producteurs (GMP) Veuillez compléter pour chaque installation raccordée le « III. Informations sur le Producteur ».


2024-08-28 14:24:30.112 | DEBUG    | wattelse.chatbot.backend.rag_backend:query_rag:371 - Calling RAG chain for question : "Quelle est la demande en puissance du client ? Quelle est la puissance maximale de soutirage demandée ?"...
2024-08-28 14:24:30.131 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:38 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2024-08-28 14:24:30.133 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:39 - Computing embeddings...
2024-08-28 14:24:30.222 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:43 - Computing embeddings done



****************************************************************************************************
ANSWER

La adresse du site de l'installation à raccorder est : 5 PLACE DES COUREAULEURS, 17000 LA ROCHELLE.
****************************************************************************************************
Document 1:

] °SIRET Type d’installation Production ou Consommation Type de modification Préciser la modification envisagée Par exemple : ajout d’une nouvelle unité de production, modification d’une unité de production existante,… Adresse Numéro / rue Code postal/ ville II- DONNEES TECHNIQUES AU POINT DE RACCORDEMENT Valeur (unité) Catég orie Statut ferme ou révisable Complément d’informations Caractéristiques essentielles de la Demande Date souhaitée de mise à disposition du raccordement MM/AAAA Date souhaitée de mise en service de l’installation de production JJ/MM/AAAA Emplacement du point de raccordement Joindre un extrait cadastral des parcelles concernées et indiquer la pos

# Tests with context compression

In [12]:
backend.use_context_compression = True

In [13]:
query_doc(question_list, fiche_D1A)


2024-08-28 11:53:27.410 | DEBUG    | wattelse.chatbot.backend.rag_backend:query_rag:371 - Calling RAG chain for question : "Quel est le nom du client ?"...
2024-08-28 11:53:27.454 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:38 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2024-08-28 11:53:27.460 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:39 - Computing embeddings...
2024-08-28 11:53:27.628 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:43 - Computing embeddings done
2024-08-28 11:53:30.460 | DEBUG    | wattelse.chatbot.backend.rag_backend:query_rag:371 - Calling RAG chain for question : "Quelle est l'adresse du site de l'installation à raccorder (extraites seulement les valeurs d'adresse) ?"...
2024-08-28 11:53:30.521 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:38 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2024-08-28 11:53:30.533


****************************************************************************************************
ANSWER

Le nom du client est European Energy France.
****************************************************************************************************
Document 1:

INFORMATIONS SUR LE DEMANDEUR DU RACCORDEMENT Nom de la société : EUROPEAN ENERGY FRANCE n° SIREN : 913794707 Nom et fonction de l’interlocuteur : Emmanuel Fourmaux - Responsable Développement Régional


2024-08-28 11:53:47.223 | DEBUG    | wattelse.chatbot.backend.rag_backend:query_rag:371 - Calling RAG chain for question : "Quelle est la demande en puissance du client ? Quelle est la puissance maximale de soutirage demandée ?"...
2024-08-28 11:53:47.257 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:38 - Calling EmbeddingAPI using model: OrdalieTech/Solon-embeddings-large-0.1
2024-08-28 11:53:47.259 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:39 - Computing embeddings...
2024-08-28 11:53:47.350 | DEBUG    | wattelse.api.embedding.client_embedding_api:embed_query:43 - Computing embeddings done



****************************************************************************************************
ANSWER

60210 SAINT-THIBAULT II.
****************************************************************************************************
Document 1:

>>>
Adresse Numéro / rue Code postal/ ville
>>>

These are the relevant parts of the context that answer the question "Quelle est l'adresse du site de l'installation à raccorder (extraites seulement les valeurs d'adresse) ?"
----------------------------------------------------------------------------------------------------
Document 2:

>> Informations sur le Producteur ». L’acte de désignation est à transmettre avec la demande de raccordement.
----------------------------------------------------------------------------------------------------
Document 3:

Extracted relevant parts:

Adresse Numéro / rue Code postal/ ville 60210 SAINT-THIBAULT II.
------------------------------------------------------------------------------------------------

In [ ]:
# https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/
# https://blog.langchain.dev/improving-document-retrieval-with-contextual-compression/